In [1]:
#instalando a biblioteca Llama index
!pip install llama-index


'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [2]:
pip install llama-index --upgrade


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip uninstall llama-index
!pip install llama-index


'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [4]:
# biblioteca para permitir a leitura de arquivos
from llama_index.core import SimpleDirectoryReader
documentos = SimpleDirectoryReader(input_dir='documentos')

In [5]:
documentos.input_files


[WindowsPath('c:/Users/santi/Desktop/Chatbot_LucasSantino/Aula7/documentos/doc_info_agricultura.pdf')]

In [6]:
docs = documentos.load_data()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)


In [7]:
docs

[Document(id_='818b7c41-94a0-441d-9c6f-71b2e26aebda', embedding=None, metadata={'page_label': '1', 'file_name': 'doc_info_agricultura.pdf', 'file_path': 'c:\\Users\\santi\\Desktop\\Chatbot_LucasSantino\\Aula7\\documentos\\doc_info_agricultura.pdf', 'file_type': 'application/pdf', 'file_size': 30901, 'creation_date': '2025-03-30', 'last_modified_date': '2025-03-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Guia de Preparo do Solo para Plantio Informações Gerais P: Qual a importância do preparo adequado do solo? R: O preparo adequado do solo garante que as plantas tenham acesso aos nutrientes necessários, além de melhorar a retenção 

In [8]:
len(docs)

3

In [9]:
# importando a biblioteca para fazer a parse das informações
from llama_index.core.node_parser import SentenceSplitter

node_parser= SentenceSplitter(chunk_size=1200)

nodes = node_parser.get_nodes_from_documents(docs,show_progress=True)

c:\Users\santi\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes: 100%|██████████| 3/3 [00:00<00:00, 360.91it/s]


In [10]:
nodes



[TextNode(id_='40f541b9-1227-498f-8997-8d48b70c129e', embedding=None, metadata={'page_label': '1', 'file_name': 'doc_info_agricultura.pdf', 'file_path': 'c:\\Users\\santi\\Desktop\\Chatbot_LucasSantino\\Aula7\\documentos\\doc_info_agricultura.pdf', 'file_type': 'application/pdf', 'file_size': 30901, 'creation_date': '2025-03-30', 'last_modified_date': '2025-03-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='818b7c41-94a0-441d-9c6f-71b2e26aebda', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'doc_info_agricultura.pdf', 'file_path': 'c:\\Users\\santi\\Desktop\\Chatbot_LucasSantino\\Aula7\\documentos\\doc_info_agricultura.pdf', 'file_type': 'application/pdf', 'file_size'

In [11]:
len(nodes)

3

In [12]:

#instalaçao do hugging face
!pip install -q llama-index-embeddings-huggingface --upgrade

'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [13]:
!pip install transformers


'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [15]:
#importa a biblioteca para fazer o embeding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

ModuleNotFoundError: No module named 'llama_index.embeddings.huggingface'

In [ ]:
#importa a biblioteca para fazer o embeding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

ModuleNotFoundError: No module named 'llama_index.embeddings.huggingface'

In [ ]:
# Classe personalizada para integrar ao Chroma DB

class ChromaEmbeddingWrapper:
  def __init__(self,model_name:str): # inicializa com o modelo embedding
    self.model = HuggingFaceEmbedding(model_name = model_name)


  def __call__(self,input): # Converte a entrada para o formato compativel com o HuggingFace
    return self.model.embed(input)


In [ ]:
embed_model_chroma= ChromaEmbeddingWrapper(model_name='intfloat/multilingual-e5-large')

NameError: name 'HuggingFaceEmbedding' is not defined

In [ ]:
#instalaçao do chroma db
!pip install -q llama-index-vector-stores-chroma

'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [ ]:
#importando chroma db

import chromadb
db = chromadb.PersistentClient(path='./chroma_db')
chroma_client = db
collection_name = 'documentos_doc_info_agricultura'

try:
 chroma_collection = chroma_client.get_or_create_collection(
        name=collection_name,
        embedding_function=embed_model_chroma)

except Exception as e :
  print(f'Erro ao criar coleçao {e}')

ModuleNotFoundError: No module named 'chromadb'

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext


ModuleNotFoundError: No module named 'llama_index.vector_stores'

In [ ]:
#criando a coleçao do banco de dados
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

NameError: name 'ChromaVectorStore' is not defined

In [ ]:
embed_model = HuggingFaceEmbedding(model_name='intfloat/multilingual-e5-large')

NameError: name 'HuggingFaceEmbedding' is not defined

In [ ]:
from llama_index.core import VectorStoreIndex

In [ ]:
index = VectorStoreIndex(nodes, storage_context=storage_context,embed_model = embed_model)

In [ ]:
from llama_index.core import load_index_from_storage
index = load_index_from_storage(storage_context,embed_model = embed_model)

In [ ]:
from google.colab import userdata
Groq_api =userdata.get('api_aula07')

ModuleNotFoundError: No module named 'google'

In [ ]:
!pip install -q llama-index-llms-groq

In [ ]:
from llama_index.llms.groq import Groq

In [ ]:
llms = Groq(model = 'llama3-70b-8192',api_key=Groq_api )

In [ ]:
chat_engine = index.as_chat_engine(mode='context',llm = llms)

In [ ]:

chat_engine.chat_history

In [ ]:
from llama_index.core.memory import ChatSummaryMemoryBuffer

In [ ]:
memory = ChatSummaryMemoryBuffer(llm=llms,token_limit=256)

In [ ]:
chat_engine= index.as_chat_engine(
    chat_mode = 'context',
    llm=llms,
    memory=memory,
    system_prompt=('''Voce é especialista em agriculruta da Agrotrack, uma emoresa no ramo de agricultura,
sua funçao é tirar duvidas de forma simpatica e natural sobre tipos de plantio, sementes e o que mais envolver a agricultura.'''))

In [ ]:
response = chat_engine.chat('Ola')
print(response)

In [ ]:
response = chat_engine.chat('Voce pode me dar mais detalhes sobre o solo?')
print(response)

In [ ]:
response = chat_engine.chat('Quais sãos os seus tipos?')
print(response)

In [ ]:
memory.get()

In [ ]:
#resetando o chat
chat_engine.reset()

In [ ]:
response = chat_engine.chat('Quais sãos os seus tipos?')
print(response)

In [ ]:
# implementando a interface com o Gradio
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
# Função para conversar com o chatbot
def converse_com_bot(message, chat_history):
    # Simulando uma resposta do chatbot (substitua com sua lógica real de chatbot)
    response = chat_engine.chat(message)  # Altere conforme necessário

    if chat_history is None:
        chat_history = []

    chat_history.append({"role": "user", "content": message})
    chat_history.append({"role": "assistant", "content": response.response})  # Ajuste conforme necessário

    return "", chat_history

# Função para resetar o chatbot
def resetar_chat():
    chat_history = []  # Reseta o histórico
    return chat_history

In [ ]:
# Criando a interface Gradio
with gr.Blocks() as app:
    gr.Markdown('# Chatbot da AgroTrack')
    chatbot = gr.Chatbot(type='messages')
    msg = gr.Textbox(label='Digite a sua mensagem')
    limpar = gr.Button('Limpar')

    msg.submit(converse_com_bot, [msg, chatbot], [msg, chatbot])
    limpar.click(resetar_chat, None, chatbot, queue=False)

    app.launch(debug=True)